In [ ]:
#import modules
import pandas as pd # for dataframes
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
import datetime as dt
import numpy as np

#split training set and test set
# from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split

# import model
from sklearn.linear_model import LinearRegression

from sklearn import metrics
import pickle

In [ ]:
data = pd.read_excel("Online_Retail.xlsx")

In [ ]:
data.head()

In [ ]:
filtered_data = data[['Country', 'CustomerID']].drop_duplicates()

In [ ]:
filtered_data

In [ ]:
#Top ten country's customer
filtered_data.Country.value_counts()[:10].plot(kind='bar')

In [ ]:
uk_data = data[data.Country=='United Kingdom']

In [ ]:
uk_data

In [ ]:
uk_data.describe()

In [ ]:
uk_data = uk_data[uk_data.Quantity>0]
# or uk_data = uk_data[(uk_data['Quantity']>0)] 

In [ ]:
uk_data

In [ ]:
uk_data.info()

In [ ]:
uk_data = uk_data[['CustomerID', 'InvoiceDate', 'InvoiceNo', 'Quantity', 'UnitPrice']]

In [ ]:
uk_data

In [ ]:
#Calulate total purchase
uk_data['TotalPurchase'] = uk_data['Quantity']*uk_data['UnitPrice']

In [ ]:
uk_data_group = uk_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda date: (date.max() - date.min()).days,
    'InvoiceNo': lambda num: len(num),
    'Quantity': lambda quan: quan.sum(),
    'TotalPurchase': lambda price: price.sum()
})

In [ ]:
uk_data_group.head()

In [ ]:
# Change the name of columns
uk_data_group.columns=['num_days','num_transactions','num_units','spent_money']
uk_data_group.head()

# Calculate CLTV using following formula
CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.

Customer Value = Average Order Value * Purchase Frequency

## 1. Calculate Average Order Value

In [ ]:
# Average Order Value
uk_data_group['avg_order_value']=uk_data_group['spent_money']/uk_data_group['num_transactions']

In [ ]:
uk_data_group.head()

# 2. Calculate Purchase Frequency

In [ ]:
# uk_data_group.shape[0]
# 3921

In [ ]:
purchase_frequency=sum(uk_data_group['num_transactions'])/uk_data_group.shape[0]

In [ ]:
purchase_frequency

## 3. Calculate Repeat Rate and Churn Rate

In [ ]:
# Repeat Rate
repeat_rate=uk_data_group[uk_data_group.num_transactions > 1].shape[0]/uk_data_group.shape[0]

In [ ]:
#Churn Rate
churn_rate=1-repeat_rate

In [ ]:
purchase_frequency,repeat_rate,churn_rate

## 4. Calculate Profit Margin
Profit margin is the commonly used profitability ratio. It represents how much percentage of total sales has earned as the gain. Let's assume our business has approx 5% profit on the total sale.

In [ ]:
# Profit Margin
uk_data_group['profit_margin']=uk_data_group['spent_money']*0.05

In [ ]:
uk_data_group.head()

## 5. Calculate Customer Lifetime Value

In [ ]:
# Customer Value
uk_data_group['CLV']=(uk_data_group['avg_order_value']*purchase_frequency)/churn_rate

In [ ]:
#Customer Lifetime Value
uk_data_group['cust_lifetime_value']=uk_data_group['CLV']*uk_data_group['profit_margin']

In [ ]:
uk_data_group.head()

# Prediction Model for CLTV

In [ ]:
uk_data.head()

In [ ]:
uk_data['month_yr'] = uk_data['InvoiceDate'].apply(lambda x: x.strftime('%b-%Y'))

In [ ]:
uk_data.head()

In [ ]:
sale=uk_data.pivot_table(index=['CustomerID'],columns=['month_yr'],values='TotalPurchase',aggfunc='sum',fill_value=0).reset_index()

In [ ]:
sale

In [ ]:
sale['CLV']=sale.iloc[:,2:].sum(axis=1)

In [ ]:
sale

## Selecting Feature
Here, you need to divide the given columns into two types of variables dependent(or target variable) and independent variable(or feature variables). Select latest 6 month as independent variable

In [ ]:
X=sale[['Dec-2011','Nov-2011', 'Oct-2011','Sep-2011','Aug-2011','Jul-2011']]
y=sale[['CLV']]

In [ ]:
X

In [ ]:
y

## Splitting Data

In [ ]:
#split training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [ ]:
# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_pred = linreg.predict(X_test)

In [ ]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

# How Well Does the Model Fit the data?
In order to evaluate the overall fit of the linear model, we use the R-squared value. R-squared is the proportion of variance explained by the model. Value of R-squared lies between 0 and 1. Higher value or R-squared is considered better because it indicates the larger variance explained by the model.

In [ ]:
from sklearn import metrics
# compute the R Square for model
print("R-Square:",metrics.r2_score(y_test, y_pred))

## Model Evaluation
For regression problems following evaluation metrics used (Ritchie Ng):

Mean Absolute Error (MAE) is the mean of the absolute value of the errors.
Mean Squared Error (MSE) is the mean of the squared errors.
Root Mean Squared Error (RMSE) is the square root of the mean of the squared errors.

In [ ]:
# calculate MAE using scikit-learn
print("MAE:",metrics.mean_absolute_error(y_test,y_pred))

#calculate mean squared error
print("MSE",metrics.mean_squared_error(y_test, y_pred))
# compute the RMSE of our predictions
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
# save the model to disk
filename = 'CLTV_model.sav'
pickle.dump(linreg, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open('CLTV_model.sav', 'rb'))

In [ ]:
customer_predict = uk_data.loc[1:1 ,:]
customer_predict

In [ ]:
customer_predict_X = X.loc[1:1,:]
customer_predict_Y = y.loc[1:1, :]

In [ ]:
result = loaded_model.score(customer_predict_X, customer_predict_Y)
print(result)